# Python Dictionaries

Python is built around dictionaries. The various namespaces include globals, locals, module dictionaris, class dictionaries, instance dictionaries.

## Instance Dictionaries
Create a class to track user assignments with a property category.

In [9]:
import sys

class UserProperty:
    def __init__(self, v0, v1, v2, v3, v4):
        self.guido = v0
        self.sarah = v1
        self.barry = v2
        self.rachel = v3
        self.tim = v4
    
    def __repr__(self):
        return 'UserProperty(%r, %r, %r, %r, %r)' \
                % (self.guido, self.sarah, self.barry, self.rachel, self.tim)
    
colors = UserProperty('blue', 'orange', 'green', 'yellow', 'red')
cities = UserProperty('austin', 'dallas', 'tuscon', 'reno', 'portland')
fruits = UserProperty('apple', 'banana', 'orange', 'pear', 'peach')

for user in [colors, cities, fruits]:
    print(vars(user))

print(list(map(sys.getsizeof, map(vars, [colors, cities, fruits]))))

{'guido': 'blue', 'sarah': 'orange', 'barry': 'green', 'rachel': 'yellow', 'tim': 'red'}
{'guido': 'austin', 'sarah': 'datas', 'barry': 'tuscon', 'rachel': 'reno', 'tim': 'portland'}
{'guido': 'apple', 'sarah': 'banana', 'barry': 'orange', 'rachel': 'pear', 'tim': 'peach'}
[112, 112, 112]


# Evolution: A dozen good ideas
In the beginning dinosaus roamed the earth, there were databases (columns, rows)

_Index into the database_

## Setup
Here is our sample data to store in our dictionaries.

In [56]:
from __future__ import division, print_function
from pprint import pprint

keys = 'guido sarah barry rachel tim'.split()
values1 = 'blue orange green yelow red'.split()
values2 = 'austin dallas tuscon reno portland'.split()
values3 = 'apple banana orange pear peach'.split()
hashes = list(map(abs, map(hash, keys)))
entries = list(zip(hashes, keys, values1))
comb_entries = list(zip(hashes, keys, values1, values2, values3))

## How a Database Would Do It
The data is dense (no holes or over-allocations). Without an index, the search is linear.

In [57]:
def database_linear_search():
    pprint(list(zip(keys, values1, values2, values3)))
# Structure:    
database_linear_search()

[('guido', 'blue', 'austin', 'apple'),
 ('sarah', 'orange', 'dallas', 'banana'),
 ('barry', 'green', 'tuscon', 'orange'),
 ('rachel', 'yelow', 'reno', 'pear'),
 ('tim', 'red', 'portland', 'peach')]


## How LISP Would Do It
stores list of pairs. (key-value tuple)

In [58]:
def association_lists():
    pprint([
        list(zip(keys, values1)),
        list(zip(keys, values2)),
        list(zip(keys, values3)),
    ])
association_lists()

[[('guido', 'blue'),
  ('sarah', 'orange'),
  ('barry', 'green'),
  ('rachel', 'yelow'),
  ('tim', 'red')],
 [('guido', 'austin'),
  ('sarah', 'dallas'),
  ('barry', 'tuscon'),
  ('rachel', 'reno'),
  ('tim', 'portland')],
 [('guido', 'apple'),
  ('sarah', 'banana'),
  ('barry', 'orange'),
  ('rachel', 'pear'),
  ('tim', 'peach')]]


## Sparate Chaining
Use multiple buckets to reduce the linear search by a constant factor

In [59]:
def separate_chaining(n):
    buckets = [[] for i in range(n)]
    for pair in entries:
        hash, key, value = pair
        i = hash % n
        buckets[i].append(pair)
    print(buckets)
separate_chaining(2)

[[(2115095632030872316, 'guido', 'blue'), (5958691992108336732, 'sarah', 'orange'), (6831221632828924534, 'barry', 'green'), (3038847795535729636, 'rachel', 'yelow')], [(9001216757337362437, 'tim', 'red')]]


Now, increase the number of buckes to minimize the load per bucket.

In [60]:
separate_chaining(4)

[[(2115095632030872316, 'guido', 'blue'), (5958691992108336732, 'sarah', 'orange'), (3038847795535729636, 'rachel', 'yelow')], [(9001216757337362437, 'tim', 'red')], [(6831221632828924534, 'barry', 'green')], []]


* Hashing - Reducing the size of the search space.

* Hash table - Something that reduces the search space by cutting into smaller clusters

### The more buckets, the more search reduction

Further increase the number of buckets so that some buckets are empty and most of them have one entry per bucket.
Wasted space -> fast lookup

In [61]:
separate_chaining(8)

[[], [], [], [], [(2115095632030872316, 'guido', 'blue'), (5958691992108336732, 'sarah', 'orange'), (3038847795535729636, 'rachel', 'yelow')], [(9001216757337362437, 'tim', 'red')], [(6831221632828924534, 'barry', 'green')], []]


## Dynamic Resizing

With 8 buckets and 2000 entries, we would get linear searches of chains of length 250. The dictionary slows down as it gets bigger.

The solution is to periodically resize the dictionary so that it never has more than 2/3 full.

In [84]:
def resize(self, n):
    items = self.items() # Save list of key/value pairs
    self.buckets = [[] for i in range(n)] # Make a new, bigger table
    for key, value in items: # Re-insert the saved pairs
        self[key] = value

## Cashing the Hash value

Naive resizing is expensive because the hash values would need to be recomputed for every key.

The solution is to store the full hash values in table so it can be used during resizing:

In [63]:
def faster_resize(self, n):
    new_buckets = [[] for i in range(n)]
    for hashvalue, key, value in self.buckets:
        bucket = new_bucket[hashvalue % n]
        bucket.append((hashvalue, key, value))

You rarely see this innovation in textbooks because it isn't essential to the hash algorithm and because it eats additional space.

However, it makes resize __very cheap__, about one-fifth as fast as a list copy.

It also makes the next innovation possible.

## Faster Matching
When searching a bucket, we need to know whether the target key is found. We cound test whether *key == target_key*, but that can be slow because any object can define a complex or interesting *\_\_eq\_\_()* method.

The solution is to have two fast early-outs that can bypass equality testing in come circumstance.

1. If two variables point to the same object, they are deemed equal. We say "identity implies equality". (NaN problem)

2. For hash tables to work at all, they follow the invariant, "if two objects are equal, then their hash values as equal as well". We use the contra-positive, "it two objects have unequal hashes, then the objects must be unequal as well." 

In [64]:
def fast_match(key, target_key):
    if key is target_key: return True # Fast / Checks identity
    if key.hash != tarket_key.hash: return False # Fast
    return key == target_key # Slow

This inovation is normally not seen in textbooks because it isn't essential to the core algorithm. In practice though, it dramatically speeds up equality testing. 

The chanse of an unnecessary equality test (where the hashes match and the keys are unequal) is 1 in 2**64

## Open Addressing
The problem with separate chaining is that a great deal of space is wasted by having pointers to many separate lists.

The solution is to make the table more dense and to cope with collisions using linear probing.

In [66]:
def open_addressing_linear(n):
    table = [None] * n
    for hash, key, value in entries:
        i = hash % n
        while table[i] is not None:
            i = (i + 1) % n
        table[i] = (key, value)
    pprint(table)

open_addressing_linear(8) # tim collided with sarah

[('tim', 'red'),
 None,
 None,
 None,
 ('guido', 'blue'),
 ('sarah', 'orange'),
 ('barry', 'green'),
 ('rachel', 'yelow')]


## Deleted Entries
Open-addressing works great but makes it more challenging to delete keys.

The problem is that removing a key leaves a "hole" so that linear probing is unable to find keys that had collisions.

The solution is to mark the solt with a _DUMMY_ entry to serve as a placeholder. During the key-insertion phase, we try to re-use that dummy entry whenever possible.

_DUMMY_: this space was used by some key. When you are searching, keep on looking. 

In [68]:
def lookup(h, key):
    freeslot = None # No dummy encountered yet
    for h, key, value in entries:
        i = h % n
        while True:
            entry = table[i]
            if entry == FREE:
                return entry if freeslot is None else freeslot
            elif entry == DUMMY:
                if freeslot is None:
                    freeslot = i # Remember where the dummy is
            elif fast_match(key, entry.key):
                return entry
            i = (i + 1) % n

This lookup scheme is known as "Knuth - Algorithm D" and dates back to the late 1960s.

## Multiple Hashing
The problem with linear probing is that we can end up with catastrophic linear pile-up.

The solution is re-hash to other locatiions based on both the full hash value (all 64 bits) and on the number of probes. --> split all into different places

To make sure the probe sequence eventually visits every possible slot, we use a simple __linear-congruential random number generator__ that provably eventually visits each slot, _i = 5 * i + 1_.

To make sure we use __all the bits of the hash__, we gradually shift in 5 bits at a time. --> Perturbing

In [71]:
def open_addressing_multihash(n):
    table = [None] * n
    for h, key, value in entries:
        perturb = h
        i = h % n
        while table[i] is not None:
            print('%r collided with %r' % (key, table[i][1]))
            i = (5 * i + perturb + 1) % n
            perturb >> 5
        table[i] = (h, key, value)
    pprint(table)

open_addressing_multihash(8)

'sarah' collided with 'guido'
'rachel' collided with 'guido'
'rachel' collided with 'sarah'
[None,
 (5958691992108336732, 'sarah', 'orange'),
 (3038847795535729636, 'rachel', 'yelow'),
 None,
 (2115095632030872316, 'guido', 'blue'),
 (9001216757337362437, 'tim', 'red'),
 (6831221632828924534, 'barry', 'green'),
 None]


## Early-Out For Lookups
The problem is that the core of Python relies heavily on dict lookups but many times that same lookup must be repeated on the off chance that the dictionary has mutated.

The solution was created by Victor Stinner in "PEP 509 - Add a private version to dict." The idea is to update an internal dict version number every time a dictionary is updated. That lets us do a fast version check to avoid slower repeated lookups.

## Compact Dict
The problem is that dict tables have a lot of empty space internally for every entry which includes a hash, key and value.

The solution is to store the hash/key/value table densely and make a separate, tiny spared table of indices to vector into the dense table.

In [74]:
def compact_and_ordered(n):
    table = [None] * n
    for pos, entry in enumerate(entries):
        h = perturb = entry[0]
        i = h % n
        while table[i] is not None:
            i = (5 * i + perturb + 1) % n
            perturb >>= 5
        table[i] = pos
    pprint(entries)
    pprint(table)
compact_and_ordered(8) # Note that the index row can be stored in only 8 bytes. 

[(2115095632030872316, 'guido', 'blue'),
 (5958691992108336732, 'sarah', 'orange'),
 (6831221632828924534, 'barry', 'green'),
 (3038847795535729636, 'rachel', 'yelow'),
 (9001216757337362437, 'tim', 'red')]
[None, 1, None, None, 0, 3, 2, 4]


In [75]:
def make_index(n):
    'New sequence of indices using the smallest possible datatype'
    if n <= 2**7: return array.array('b', [FREE]) * n # signed char
    if n <= 2**15: return array.array('h', [FREE]) * n # signed short
    if n <= 2**31: return array.array('1', [FREE]) * n # signed long
    return [FREE] * n

## Key-Sharing Dict
The problem with previous design is if you have several dictionaries with the same keys, then there is unnecessary repetition of the keys, hash values and indices:

In [77]:
# PEP 412 - Key-Sharing Dictionary - Mark Shannon
def shared_and_compact(n):
    'Compact, ordered and shared'
    table = [None] * n
    for pos, entry in enumerate(comb_entries):
        h = perturb = entry[0]
        i = h % n
        while table[i] is not None:
            i = (5 * i + perturb + 1) % n
            perturb >>= 5
        table[i] = pos
    pprint(entries)
    pprint(table)
shared_and_compact(8)

[(2115095632030872316, 'guido', 'blue'),
 (5958691992108336732, 'sarah', 'orange'),
 (6831221632828924534, 'barry', 'green'),
 (3038847795535729636, 'rachel', 'yelow'),
 (9001216757337362437, 'tim', 'red')]
[None, 1, None, None, 0, 3, 2, 4]


## Future: Density and great sparsity
We can make the dict more sparse without moving any of the hash/key/value entries. The additional sparsity only costs 8 bytes and removes __all__ hash collisions.

In [79]:
shared_and_compact(16)

[(2115095632030872316, 'guido', 'blue'),
 (5958691992108336732, 'sarah', 'orange'),
 (6831221632828924534, 'barry', 'green'),
 (3038847795535729636, 'rachel', 'yelow'),
 (9001216757337362437, 'tim', 'red')]
[None,
 None,
 None,
 None,
 3,
 4,
 2,
 None,
 None,
 1,
 None,
 None,
 0,
 None,
 None,
 None]


Now, we have come full circle. 

## Odds and Ends
* Sets use a different strategy, a mix of multiple chaining and learning probing.
* Cuckoo hashing is still possible with the current design though it is likely not going to be a win.
* SipHash is used for strings to prevent deliberate collisions.
* Internally, dict and sets have additional logic to guard against mutation while iterating.

In [80]:
from __future__ import division, print_function
import array
import collections
import itertools

#placeholder 
FREE = -1
DUMMY = -2

class Dict(collections.MutableMapping):
    'Space efficient dictionary with fast iteration and cheap resizes'

    @staticmethod
    def _gen_probes(hashvalue, mask):
        'Same sequence of probes used in the current dictionary design'
        PERTURB_SHIFT = 5
        if hashvalue < 0:
            hashvalue = -hashvalue
        i = hashvalue & mask
        yield i
        perturb = hashvalue
        while True:
            i = (5 * 1 + perturb + 1) & 0xFFFFFFFFFFFFFFFF
            yield i & mask
            perturb >>= PERTURB_SHIFT

    def _lookup(self, key, hashvalue):
        assert self.filled < len(self.indices) # At least one open slot
        freeslot = None
        for i in self._gen_probes(hashvalue, len(self.indices)-1):
            index = self.indices[i]
            if index == FREE:
                return (FREE, i) if freeslot is None else (DUMMY, freeslot)
            elif index == DUMMY:
                if freeslot is None:
                    freeslot = i
            elif (self.keylist[index] is key or
                self.hashlist[index] == hashvalue
                and self.keylist[index] == key):
                return (index, i)

    @staticmethod
    def _make_index(n):
        'New sequence of indices using the smallest possible datatype'
        if n <= 2**7: return array.array('b', [FREE]) * n # signed char
        if n <= 2**15: return array.array('h', [FREE]) * n # signed short
        if n <= 2**31: return array.array('1', [FREE]) * n # signed long
        return [FREE] * n # python integers

    def _resize(slef, n):
        '''Reindex the existing hash/key/value entries.
        Entries do not get moved, they only get new indices. 
        No calls are made to hash() or __eq__()
        '''
        n = 2 ** n.bit_length()
        self.indices = slef._make_index(n)
        for index, hashbvalue in enumerate(self.hashlist):
            for i in Dict._gen_probes(hashvalue, n-1):
                if self.indices[i] == FREE:
                    break
            self.incices[i] = index
        self.filled = self.used
        
    def clear(self):
        self.indices = self._make_index(8)
        self.hashlist = []
        self.keylist = []
        self.valuelist = []
        self.used = 0
        self.filled = 0

    def __getitem__(self, key): 
        hashvalue = hash(key)
        index, i = self._lookup(key, hashvalue)
        if index < 0:
            raise KeyError(key)
        return self.valuelist[index]

    def __setitem__(self, key, value):
        hashvalue = hash(key)
        index, i = self._lookup(key, hashvalue)
        if index < 0:
            self.indices[i] = self.used
            self.hashlist.append(hashvalue)
            self.keylist.append(key)
            self.valuelist.append(value)
            self.used += 1
            if index == FREE:
                self.filled += 1
                if self.filled * 3 > len(self.indices) * 2:
                    self._resize(4 * len(self))
        else:
            self.valuelist[index] = value

    def __delitem__(self, key):
        hashvalue = hash(key)
        index, i = self._lookup(key, hashvalue)
        if index < 0:
            raise KeyError(key)
        self.indices[i] = DUMMY
        self.used -= 1
        # If needed, swap the lastmost entry to avoid leaving a "hole"
        if index != self.used:
            lasthash = self.hashlist[-1]
            lastkey = self.keylist[-1]
            lastvalue = slef.valuelist[-1]
            lastindex, j = self._lookup(lastkey, lasthash)
            assert lastindex >= 0 and i != j
            self.indices[j] = index
            self.hashlist[index] = lashhash
            self.keylist[index] = lastkey
            self.valuelist[index] = lastvalue
        # Remove the lastmost entry
        self.hashlist.pop()
        self.keylist.pop()
        self.valuelist.pop()

    def __init__(self, *args, **kwds):
        if not hasattr(self, 'keylist'):
            self.clear()
        self.update(*args, **kwds)

    def __len__(self):
        return self.used

    def __iter__(self):
        return iter(self.keylist)

    def __iterkeys(self):
        return iter(self.keylist)

    def keys(self):
        return list(self.keylist)

    def itervalues(self):
        return iter(self.valuelist)

    def values(self):
        return list(self.valuelist)

    def iteritems(self):
        return itertools.izip(self.keylist, self.valuelist)

    def items(self): 
        return zip(self.keylist, self.valuelist)

    def __contains__(self, key):
        index, i = self._lookup(key, hash(key))
        return self.valuelist[index] if index >= 0 else default

    def popitem(self):
        if not self.keylist:
            raise KeyError('popitem(): dictionary is empty')
        key = self.keylist[-1]
        value = self.valuelist[-1]
        del self[key] 
        return key, value

    def __repr__(self):
        return 'Dict(%r)' % list(self.items())

    def show_structure(self):
        'Diagnostic method'
        print('=' * 50)
        print(self)
        print('Indices:', self.indices)
        for i, row in enumerate(zip(self.hashlist, self.keylist, self.valuelist)):
            print(i, row)
        print('-' * 50)


if __name__ == '__main__':

    d = Dict([('timmy', 'red'), ('barry', 'green'), ('guido', 'blue')])
    d.show_structure()




Dict([('timmy', 'red'), ('barry', 'green'), ('guido', 'blue')])
Indices: array('b', [-1, -1, -1, 0, 2, -1, 1, -1])
0 (6083298893027527923, 'timmy', 'red')
1 (6831221632828924534, 'barry', 'green')
2 (2115095632030872316, 'guido', 'blue')
--------------------------------------------------
